# === ZeroBERTo Fit ===

# Dependencies

In [439]:
# !pip install setfit
# !pip install datasets
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [440]:
# from google.colab import drive
# import pandas as pd

# drive.mount('/content/drive')

In [441]:
import pandas as pd
import datasets_handler
import datasets
from datasets import Dataset
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer, sample_dataset
import evaluation_metrics
import gc
import zeroberto
tqdm.pandas()


# Data Prep

In [419]:
use_zeroshot_previous_step = True
exec_time = '2023_02_13__06_35_08'

# which_dataset = 'folhauol'
# which_dataset = 'bbc-news'
# which_dataset = 'ag_news'


split = "zeroshot" if use_zeroshot_previous_step else "fewshot"
n = 8
test_dataset_sample_size = 1/8
## 1/64 ---> <6 min
## 1/32 ---> <12 min
## 1/16 ---> <22 min
random_state = 422

<class 'torch.device'>


In [420]:
top_n = n

## Import Data

In [421]:
which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
# which_dataset = 'ag_news'

hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)


### Data from Zero-Shot previous step

In [422]:
# def getZeroshotPreviousData(df_results_zeroberto,top_n = 8):
#     df_results_zeroberto.index = df_results_zeroberto['Unnamed: 0'] ### recover original indexes for dataset
#     df_top_n = df_results_zeroberto.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)
#     df_top_n = df_top_n.drop(columns=["Unnamed: 0",class_col,class_col+"_code"])
#     return df_top_n


In [431]:
if use_zeroshot_previous_step == True:
    zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(which_dataset,class_col,top_n=top_n,exec_time=exec_time)
    raw_data_final, new_class_col = datasets_handler.mergeLabelingToDataset(raw_data,zeroshot_previous_data,class_col)
#     raw_data_final = raw_data.join(zeroshot_previous_data)
# #     ## overwrite true labels with predictions from zeroshot
#     new_class_col = 'new_'+class_col
#     raw_data_final.loc[~raw_data_final['prediction'].isna(),new_class_col] = raw_data_final['prediction'] 
#     raw_data_final.loc[raw_data_final['prediction'].isna(),new_class_col] = raw_data_final[class_col]

# #     ## keep true labels of the rest, for testing
#     raw_data_final = evaluation_metrics.Encoder(raw_data_final,[new_class_col])

if use_zeroshot_previous_step == False:
    raw_data_final = raw_data
    raw_data_final[new_class_col] = raw_data_final[class_col]



## Dataset Split

### Standard SetFit (Few-Shot)

In [432]:
### divide dataframe em treino e teste, com n samples para few-shot

if (split=="fewshot"):
  df_train = df_encoded.groupby('new_Category_code')[['full_text','new_Category_code']].apply(lambda s: s.sample(min(len(s),n)))
  keys = list(df_train.columns.values)
  i1 = df_encoded.set_index(keys).index
  i2 = df_train.set_index(keys).index
  df_test = df_encoded[~i1.isin(i2)]
  df_test = df_test.groupby('new_Category_code')[['full_text','new_Category_code']].apply(lambda x:x.sample(int(len(x)*test_dataset_sample_size)))
  df_train = df_train.astype(str)
  df_test = df_test.astype(str)

### Zeroberto SetFit

In [433]:
if (split == "zeroshot"):
  df_train = raw_data_final[~raw_data_final['prediction'].isna()].groupby(new_class_col+"_code")[[data_col,new_class_col+"_code"]].apply(lambda s: s.sample(min(len(s),top_n),random_state=random_state))

  keys = list(df_train.columns.values)

  i1 = raw_data_final.set_index(keys).index
  i2 = df_train.set_index(keys).index

  df_test = raw_data_final[~i1.isin(i2)]

  df_test = df_test.groupby(new_class_col+"_code")[[data_col,new_class_col+"_code"]].apply(lambda x:x.sample(int(len(x)*test_dataset_sample_size),random_state=random_state))

  df_train = df_train.astype(str)
  df_test = df_test.astype(str)

### Build Dataset Dict

In [434]:
### transforma dataframes em datasetdict

train_dataset = Dataset.from_dict(df_train)
test_dataset = Dataset.from_dict(df_test)
dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})
dataset = dataset_dict
dataset

DatasetDict({
    train: Dataset({
        features: ['full_text', 'new_category_code'],
        num_rows: 96
    })
    test: Dataset({
        features: ['full_text', 'new_category_code'],
        num_rows: 11513
    })
})

# SetFit

## Model

In [435]:
# model = saved_model

In [436]:
model_name = "sentence-transformers/nli-roberta-base-v2"

#### Models
# "sentence-transformers/paraphrase-mpnet-base-v2"
# "ricardo-filho/bert-base-portuguese-cased-nli-assin-2"
# "ricardo-filho/bert-portuguese-cased-nli-assin-2"
# "sentence-transformers/nli-roberta-base-v2"
# "neuralmind/bert-large-portuguese-cased"
# "joeddav/xlm-roberta-large-xnli"
# "openai-gpt"

model = SetFitModel.from_pretrained(model_name)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


## Parameters

In [437]:
%%time

batch_size = 8
num_text_pairs = 20
num_epochs = 1

setfit_config = {
    "model":model_name,
    "dataset":which_dataset,
    "batch_size":batch_size,
    "num_pairs":num_text_pairs,
    "num_epochs":num_epochs,
    "dataset_sample_size":test_dataset_sample_size,
    "N_examples":n,
    "split":split,
    "0shot_data":exec_time
}

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=setfit_config["batch_size"],
    num_iterations=setfit_config["num_pairs"], # Number of text pairs to generate for contrastive learning
    num_epochs=setfit_config["num_epochs"], # Number of epochs to use for contrastive learning
    column_mapping = {data_col: "text", new_class_col+"_code": "label"} # NÃO mudar
)

print(setfit_config)

{'model': 'sentence-transformers/nli-roberta-base-v2', 'dataset': 'folhauol', 'batch_size': 8, 'num_pairs': 20, 'num_epochs': 1, 'dataset_sample_size': 0.125, 'N_examples': 8, 'split': 'zeroshot', '0shot_data': '2023_02_13__06_35_08'}
CPU times: user 103 µs, sys: 119 µs, total: 222 µs
Wall time: 201 µs


## Training

In [438]:
%%time
trainer.train()
gc.collect()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 3840
  Num epochs = 1
  Total optimization steps = 480
  Total train batch size = 8


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/480 [00:00<?, ?it/s]

CPU times: user 31min 51s, sys: 29min 35s, total: 1h 1min 26s
Wall time: 24min 1s


157

## Inference

In [442]:
%%time
y_pred = zeroberto.getPredictions(trainer)
#### folhauol: aprox. 4 predicoes por segundo
#### bbcnews: aprox. 4 preds por segundo
#### ag_news: aprox. 33 preds por segundo

Running predictions on 11513 sentences.
CPU times: user 15min 18s, sys: 1min 20s, total: 16min 39s
Wall time: 5min 48s


# Evaluation

## Metrics

In [443]:
try: print(all_metrics)
except: pass
all_metrics = evaluation_metrics.get_metrics(y_pred ,test_dataset[new_class_col+"_code"])
print(all_metrics)

{'weighted': [{'accuracy': 0.20114149523331662}, {'precision': 0.22881146136530464}, {'recall': 0.20114149523331662}, {'f1': 0.18090418710811065}], 'macro': [{'accuracy': 0.20114149523331662}, {'precision': 0.22881431442863304}, {'recall': 0.20116664800160183}, {'f1': 0.18091511211537042}]}
{'weighted': [{'accuracy': 0.08625032571875271}, {'precision': 0.2665887029667002}, {'recall': 0.08625032571875271}, {'f1': 0.11580953355643005}], 'macro': [{'accuracy': 0.08625032571875271}, {'precision': 0.11373125751651995}, {'recall': 0.07560143115044159}, {'f1': 0.06135196748819156}]}


## Save Results

In [444]:
setfit_exec_time  = evaluation_metrics.saveResults(setfit_config,all_metrics)
print(setfit_config)

metrics_setfit_2023_02_13__15_39_50.csv
config_setfit_2023_02_13__15_39_50.csv
{'model': 'sentence-transformers/nli-roberta-base-v2', 'dataset': 'folhauol', 'batch_size': 8, 'num_pairs': 20, 'num_epochs': 1, 'dataset_sample_size': 0.125, 'N_examples': 8, 'split': 'zeroshot', '0shot_data': '2023_02_13__06_35_08'}


# Save Model

In [380]:
model_path = "/Users/alealcoforado/Documents/Projetos/Modelos/{exec}".format(exec=setfit_exec_time)
print(model_path)
trainer.model._save_pretrained(save_directory=model_path)

/Users/alealcoforado/Documents/Projetos/Modelos/2023_02_13__03_29_16


## Load Model

In [16]:
load_path = r'/Users/alealcoforado/Documents/Projetos/Modelos/setfit_top_n=4_n=8_15pairs_v1'

saved_model = SetFitModel._from_pretrained(load_path)
model_name = load_path

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'weighted': [{'accuracy': 0.7208791208791209},
  {'precision': 0.7966533533828771},
  {'recall': 0.7208791208791209},
  {'f1': 0.7383634133416869}],
 'macro': [{'accuracy': 0.7208791208791209},
  {'precision': 0.47294791260373964},
  {'recall': 0.6956977371368128},
  {'f1': 0.5148314501574386}]}

  {'weighted': [{'accuracy': 0.7088331515812432},
  {'precision': 0.8112290505989518},
  {'recall': 0.7088331515812432},
  {'f1': 0.7416526123050681}],
 'macro': [{'accuracy': 0.7088331515812432},
  {'precision': 0.5086466490124514},
  {'recall': 0.7260511792805407},
  {'f1': 0.5487853669870654}]}

1/4 dataset
  {'weighted': [{'accuracy': 0.712040293517433},
  {'precision': 0.7969596487586078},
  {'recall': 0.712040293517433},
  {'f1': 0.7365853189562236}],
 'macro': [{'accuracy': 0.712040293517433},
  {'precision': 0.4964842305418267},
  {'recall': 0.7028323441240695},
  {'f1': 0.5374745594204607}]}

full dataset
{'weighted': [{'accuracy': 0.7090563785338325},
  {'precision': 0.7967113724849181},
  {'recall': 0.7090563785338325},
  {'f1': 0.734316306685027}],
 'macro': [{'accuracy': 0.7090563785338325},
  {'precision': 0.4954781686924334},
  {'recall': 0.7020811904269998},
  {'f1': 0.5360406756304892}]}